In [ ]:
import pandas as pd
import json
from openpyxl import Workbook

# Load JSON data for features and non-functional requirements
def load_features_and_non_functionalities(json_file):
    with open(json_file, 'r') as file:
        data = json.load(file)
    features = data['features']
    non_functionalities = data['non_functionalities']
    return features, non_functionalities

# Generate TTA and FIA tables
def generate_tta_fia_tables(features):
    # TTA and FIA tables
    tta_table = pd.DataFrame(index=features, columns=non_functionalities)
    fia_table = pd.DataFrame(index=features, columns=features)

    # Fill tables with "Yes" or blank based on interactions (for example, setting default as "Yes")
    tta_table.fillna("Yes", inplace=True)
    fia_table.fillna("Yes", inplace=True)

    return tta_table, fia_table

# Generate PTS sheet based on TTA and FIA
def generate_pts_sheet(tta_table, fia_table):
    pts_data = []
    pts_id_counter = 1
    
    # Get interactions marked "Yes" in TTA
    for feature in tta_table.index:
        for test_type in tta_table.columns:
            if tta_table.loc[feature, test_type] == "Yes":
                pts_data.append([f"PTS-{pts_id_counter}", f"{feature} - {test_type} interaction"])
                pts_id_counter += 1

    # Get interactions marked "Yes" in FIA
    for feature1 in fia_table.index:
        for feature2 in fia_table.columns:
            if feature1 != feature2 and fia_table.loc[feature1, feature2] == "Yes":
                pts_data.append([f"PTS-{pts_id_counter}", f"{feature1} - {feature2} interaction"])
                pts_id_counter += 1

    pts_df = pd.DataFrame(pts_data, columns=["PTS ID", "Description"])
    return pts_df

# Generate Test Case sheet template
def generate_test_case_sheet():
    test_case_columns = ["Test Case ID", "Feature", "Scenario", "Preconditions", "Steps", "Expected Result", "Actual Result", "Status"]
    test_case_df = pd.DataFrame(columns=test_case_columns)
    return test_case_df

# Main function to execute script
def main(json_file, output_file):
    # Load features and non-functional requirements
    features, non_functionalities = load_features_and_non_functionalities(json_file)

    # Generate TTA and FIA tables
    tta_table, fia_table = generate_tta_fia_tables(features)

    # Generate PTS sheet
    pts_df = generate_pts_sheet(tta_table, fia_table)

    # Generate Test Case sheet
    test_case_df = generate_test_case_sheet()

    # Create a new Excel workbook and add sheets
    with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
        tta_table.to_excel(writer, sheet_name="TTA")
        fia_table.to_excel(writer, sheet_name="FIA")
        pts_df.to_excel(writer, sheet_name="PTS", index=False)
        test_case_df.to_excel(writer, sheet_name="Test Cases", index=False)

    print(f"Excel file '{output_file}' created successfully with TTA, FIA, PTS, and Test Cases sheets.")

# Run the script with the specified JSON file and output file
if __name__ == "__main__":
    json_file = "Utilities/features.json"  # Replace with your JSON input file path
    output_file = "test_document.xlsx"  # Output Excel file path
    main(json_file, output_file)
